In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from pipelines.preprocessors import *
%load_ext autoreload
%autoreload 2

In [5]:
# Carga de los datasets
import pandas as pd

data = pd.read_parquet('../../data/raw/data_raw.parquet')
data

,_id,merchant_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type
0,bb391cb3-58cc-4b3d-a05b-069334166518,42337393,14785559,2025-01-01 00:00:34.594155,WZYM70489102961361,96a9954f-dc33-41f9-8c1f-55ab715629ee,129.25,debit
1,b3133c13-658c-4355-8bf1-5d12def38120,07726996,78149311,2025-01-01 00:00:48.609195,JVFH04190575050798,31a3ac4d-828c-47e7-87d5-b7be1d30d990,419.98,credit
2,3e7b3e3c-5a69-4d3e-8d5c-8c667379db61,02092645,49760118,2025-01-01 00:01:18.633318,YNYI72020795591498,ae27d432-1dc1-47eb-b11c-767385351a69,363.76,debit
3,dde61a05-23a9-49a3-b1d3-0b04f5aba23b,14507120,15077417,2025-01-01 00:01:21.015788,BSQS28830753307466,3ed8c56c-da09-4fa0-9282-8d8044b591f7,207.11,debit
4,f347b2f7-d832-472a-906c-699add8c0432,92639652,40915777,2025-01-01 00:02:38.592790,PKWS50530156469793,30d73df7-1c22-4cea-b355-56a5f2bd92f2,254.99,debit
...,...,...,...,...,...,...,...,...
19995,c1dd30c1-3398-461f-8fd4-d3b5fcba4280,46701411,28044710,2025-01-09 16:20:45.611470,FUJQ61156230678019,59e4b671-4774-4c58-85f8-bc16f7860b50,354.76,debit
19996,5468f38d-80f8-40af-8ebc-f284724990c6,46701411,28044710,2025-01-09 16:35:57.611470,FUJQ61156230678019,59e4b671-4774-4c58-85f8-bc16f7860b50,113.06,debit
19997,9bafe924-5da6-41ea-a187-5ae0ed0e7a58,74356706,27728710,2025-01-09 18:27:16.025029,RHHL68905810801533,493e904d-30ec-4796-a59b-457fc0d7ac73,104.17,debit
19998,5fc98d0a-eeca-43ec-ab1c-39c36549fe09,15799708,62981095,2025-01-09 18:50:45.031184,KDUZ82613506210840,5aaab32f-ce63-42b6-ab05-347fd556b37d,49.95,debit


In [6]:
# Verificar si cada account_number está asociado a un único user_id
unique_accounts = data.groupby('user_id')['account_number'].nunique()

# Analizar resultados
if unique_accounts.max() == 1:
    print('La relación es 1:1 (una cuenta por usuario).')
else:
    print('La relación no es 1:1. Hay cuentas compartidas por varios usuarios.')

La relación no es 1:1. Hay cuentas compartidas por varios usuarios.


In [7]:
# Ordenar por account_number, user_id y transaction_date para un análisis secuencial
data = data.sort_values(by=['account_number', 'user_id', 'transaction_date'])

# Crear una columna indicando la diferencia de tiempo entre transacciones consecutivas
data['windows_time'] = data.groupby(['account_number', 'user_id'])['transaction_date'].diff().dt.total_seconds().fillna(0) / 3600

# Agrupar por account_number y user_id para consolidar la información
data = data.groupby(['account_number', 'user_id']).agg(
    windows_time=('windows_time', 'sum'),
    transaction_count=('transaction_date', 'count'),
    transaction_date=('transaction_date', 'mean'),
    transaction_amount=('transaction_amount', 'sum'),
    transaction_type=('transaction_type', lambda x: x.mode()[0] if not x.mode().empty else None),  # Usamos pandas.Series.mode
).reset_index()

# Crear la columna fraction_flag basada en las reglas
data['fraction_flag'] = data.apply(
    lambda row: 'fraccionada' if row['windows_time'] <= 24 and row['transaction_count'] > 1 else 'no fraccionada',
    axis=1
)

data.drop(columns=['windows_time', 'transaction_count'], inplace=True)
data

,account_number,user_id,transaction_date,transaction_amount,transaction_type,fraction_flag
0,AABF72379081113950,1ca3c448-0279-46a6-8f97-97b06d7ce3c9,2025-01-02 09:51:51.628826880,90.50,debit,no fraccionada
1,AABL46199073805045,901bcdef-b56f-48ce-bd66-971e88476c56,2025-01-07 10:41:55.302610944,258.64,debit,no fraccionada
2,AAEG88841649433869,9f708368-cb3c-48ca-a392-57316b79afcc,2025-01-04 00:10:32.930148096,196.79,debit,no fraccionada
3,AAFH61149854104782,4c6a70f4-8ede-444d-876c-f68c3f1be0d0,2025-01-01 07:41:08.506103040,95.11,debit,no fraccionada
4,AAFJ92785234615121,8d244e3e-c4da-4ddb-a0b1-5abaa6a27967,2025-01-08 14:42:41.516628992,921.28,debit,fraccionada
...,...,...,...,...,...,...
15195,ZZVU82915110913519,f8cda88b-436d-46e2-b83c-fe0be037e5ed,2025-01-07 14:31:48.443721984,362.73,credit,no fraccionada
15196,ZZVX10748935558717,23e2fcb4-72d8-467d-894a-05e430b187ef,2025-01-02 08:19:45.090318080,314.03,debit,no fraccionada
15197,ZZWE93426747284460,497ec6d1-081f-46dc-b8d9-a88aef0bea4f,2025-01-01 01:28:22.675819008,307.59,debit,no fraccionada
15198,ZZWK36251231154111,2d174fc9-6f7c-45ea-a72a-6d8eb5122df8,2025-01-04 07:54:49.409979904,724.54,debit,fraccionada


In [8]:
# Preprocesadores
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('duplicaed_columns', DropDuplicateColumnsTransformer()),
    ('duplicated_rows', DropDuplicatedRowsTransformer()),
    ('date_columns', DateColumnsTransformer()),
    ('columns_rename', ColumnsRenameTransformer(lambda col: str(col).lower().strip().replace(' ', '_'))),
    ('nan_values', FillMissingValuesTransformer()),
    ('categorical_columns', CategoricalColumnsTransformer()),
    ('nan_columns', DropMissingValuesColumnsTransformer(threshold=1/3))
])

# Aplicar el pipeline a los datos
data_processed = pd.DataFrame(pipe.fit_transform(data))

In [11]:
# Valores faltantes
data_processed.isnull().sum()

account_number        0
user_id               0
transaction_date      0
transaction_amount    0
transaction_type      0
fraction_flag         0
dtype: int64

In [12]:
# Exportar
data_processed.to_parquet('../../data/processed/data_processed.parquet', index=False)